# Lab 3 Part I: UCI-HAR CNN with MicroAI deployment

## Imports

In [1]:
import copy
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

2023-03-24 14:06:21.536656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load and format UCI-HAR dataset (raw data)

In [2]:
dataset_path = get_file(None, "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip", extract=True, file_hash="53e099237392e0b9602f8c38f578bd8f") # Download, cache and extract UCI-HAR
dataset_dir = Path(dataset_path).parent

def load_set(dataset_dir, part: str): # Load separate sensor signals and combine them into a single array, load labels separately
    data = np.hstack([np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/'Inertial Signals'/f'{sensor}_{axis}_{part}.txt')
                for sensor in ('body_acc', 'body_gyro', 'total_acc')
                    for axis in ('x', 'y', 'z')]).reshape((-1, 128, 9))
    labels = to_categorical(np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/f'y_{part}.txt') - 1)
    return data, labels

x_train, y_train = load_set(dataset_dir, 'train')
x_test, y_test = load_set(dataset_dir, 'test')

60999314/60999314 [==============================] - 158s 3us/step


## Export complete test dataset (2947 vectors)

In [9]:
np.savetxt('x_test_uci-har.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har.csv', y_test, delimiter=',', fmt='%s')

## Export small dataset (250 vectors)

In [10]:
x_test_250 = x_test[0:250]
y_test_250 = y_test[0:250]
np.savetxt('x_test_uci-har_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [11]:
model = Sequential()
model.add(Input(shape=(128, 9)))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(filters=8, kernel_size=5, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(filters=8, kernel_size=7, activation='relu'))
model.add(Flatten())
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 126, 8)            224       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 63, 8)            0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 59, 8)             328       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 29, 8)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 23, 8)             456       
                                                                 
 flatten_1 (Flatten)         (None, 184)              

## Train model

In [12]:
history = model.fit(x_train, y_train, epochs=7, validation_data=(x_test, y_test))

Epoch 1/7
230/230 [==============================] - 2s 5ms/step - loss: 0.3866 - categorical_accuracy: 0.8481 - val_loss: 0.3242 - val_categorical_accuracy: 0.8768
Epoch 2/7
230/230 [==============================] - 1s 4ms/step - loss: 0.1760 - categorical_accuracy: 0.9329 - val_loss: 0.2745 - val_categorical_accuracy: 0.9043
Epoch 3/7
230/230 [==============================] - 1s 4ms/step - loss: 0.1381 - categorical_accuracy: 0.9464 - val_loss: 0.4041 - val_categorical_accuracy: 0.8643
Epoch 4/7
230/230 [==============================] - 1s 4ms/step - loss: 0.1410 - categorical_accuracy: 0.9472 - val_loss: 0.2040 - val_categorical_accuracy: 0.9257
Epoch 5/7
230/230 [==============================] - 1s 4ms/step - loss: 0.1113 - categorical_accuracy: 0.9566 - val_loss: 0.3641 - val_categorical_accuracy: 0.8968
Epoch 6/7
230/230 [==============================] - 1s 4ms/step - loss: 0.1140 - categorical_accuracy: 0.9543 - val_loss: 0.2654 - val_categorical_accuracy: 0.9179
Epoch 7/7


## Evaluate model on complete test dataset

In [13]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

93/93 - 0s - loss: 0.2977 - categorical_accuracy: 0.9013 - 151ms/epoch - 2ms/step
93/93 [==============================] - 0s 2ms/step
tf.Tensor(
[[464  18  13   0   1   0]
 [  3 459   5   1   3   0]
 [  9  83 328   0   0   0]
 [  0   5   0 386 100   0]
 [  0   2   0  48 482   0]
 [  0   0   0   0   0 537]], shape=(6, 6), dtype=int32)


## Evaluate model on small dataset

In [14]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 0.4764 - categorical_accuracy: 0.8320 - 27ms/epoch - 3ms/step
8/8 [==============================] - 0s 2ms/step
tf.Tensor(
[[52  1  0  0  0  0]
 [ 0 25  0  0  0  0]
 [ 0 14 10  0  0  0]
 [ 0  0  0 19 27  0]
 [ 0  0  0  0 54  0]
 [ 0  0  0  0  0 48]], shape=(6, 6), dtype=int32)


## Save trained model

In [15]:
model.save('lab3_part1_uci-har_microai.h5')

## Remove SoftMax layer

In [16]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [17]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kerascnn2c: filename=kerascnn2c-1.0.0-py3-none-any.whl size=21339 sha256=dfcf69d0b121998edefd644edf1972299c9223344be706a1e4219e84abd6c140
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ax6oj4l/wheels/29/df/9b/d62a64e871a29555dc13bc0c189d46297cdf80a3332230aaa1
Successfully built kerascnn2c


## Generate C code for the trained model with 32-bit floating-point representation

In [18]:
res = kerascnn2c.Converter(output_path=Path('uci-har_output_floating'),
                           fixed_point=0, # floating-point
                           number_type='float', # Data type for weights/activations (single-precision floating-point)
                           long_number_type='float', # Data type for intermediate results
                           number_min=-float('inf'), # Minimum value for float is negative infinity
                           number_max=float('inf') # Maximum value for float is infinity
                          ).convert_model(copy.deepcopy(model))
with open('uci-har_model_floating.h', 'w') as f:
    f.write(res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......conv1d_1
.........vars
............0
............1
......conv1d_2
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......input_layer
.........vars
......max_pooling1d
.........vars
......max_pooling1d_1
.........vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-03-24 14:13:15           64
config.json                                    2023-03-24 14:13:15         3531
variables.h5                                   2023-03-24 14:13:15        30896
Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-03-24 14:13:14           64
config.json               

## Compile the 32-bit floating-point C code for x86 and evaluate

In [22]:
!echo $PWD
!g++ -Wall -Wextra -pedantic -Ofast -o uci-har_floating -Iuci-har_output_floating/ uci-har_output_floating/model.c main.cpp
!./uci-har_floating x_test_uci-har.csv y_test_uci-har.csv

/home/jovyan/work/Lab3
uci-har_output_floating/model.c: In function ‘void cnn(const number_t (*)[128], number_t*)’:
uci-har_output_floating/model.c:104:18: warning: left operand of comma operator has no effect []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-value-Wunused-value]8;;]
  104 |     activations1.conv1d_5_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~
Testing accuracy: 0.901255


## Generate C code for the trained model with 16-bit fixed-point representation

In [20]:
res = kerascnn2c.Converter(output_path=Path('uci-har_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('uci-har_model_fixed.h', 'w') as f:
    f.write(res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......conv1d_1
.........vars
............0
............1
......conv1d_2
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......input_layer
.........vars
......max_pooling1d
.........vars
......max_pooling1d_1
.........vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-03-24 14:14:10           64
config.json                                    2023-03-24 14:14:10         3531
variables.h5                                   2023-03-24 14:14:10        30896
Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-03-24 14:14:10           64
config.json               

## Compile the 16-bit fixed-point C code for x86 and evaluate

In [21]:
!g++ -Wall -Wextra -pedantic -Ofast -o uci-har_fixed -Iuci-har_output_fixed/ uci-har_output_fixed/model.c main.cpp 
!./uci-har_fixed x_test_uci-har.csv y_test_uci-har.csv

uci-har_output_fixed/model.c: In function ‘void cnn(const number_t (*)[128], number_t*)’:
uci-har_output_fixed/model.c:104:18: warning: left operand of comma operator has no effect []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-value-Wunused-value]8;;]
  104 |     activations1.conv1d_5_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~
Testing accuracy: 0.901255
